In [21]:
#-------------------------------------------------------------------------------------------------
# BOMB-MARKETING WEB SCRAPPER
# 
# Autor: Pablo Marchesi
# Febrero 2023
#-------------------------------------------------------------------------------------------------

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

xlsx = r'C:\Users\Usuario\Downloads\tiendas de ropa_cleaned.xlsx'
df = pd.read_excel(xlsx)
web = df['website'].dropna()
email_list = []

for i in range(len(web)):
    url = web.iloc[i]
    try:
        # Extraccion del código fuente de la web 
        response = requests.get(url, timeout=2)
        soup = BeautifulSoup(response.content, "html.parser")

        # Chequea si la pagina tiene pagina de contacto (dara un aviso en caso de que no encuentre el email)
        contact_link = soup.find("a", href=re.compile("contacto|quienes somos|contactus|quiénes somos|contactanos|contactar|contáctanos|contact us|contact", re.IGNORECASE))

        # Busca los emails en el codigo fuente 
        email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[com|es|net|org|edu|gov|info|biz|io|co|uk|au|ca]{2,}" 
        emails = list(set((re.findall(email_pattern, str(soup)))))

        # Filtrado para emails muy largos
        n = 40
        emails = [x for x in emails if len(x) <= n]
        email_list.append(emails)

        # Si no ha encntrado email pero hay pagina de contacto, buscalo ahi
        if(contact_link) and emails == []: 
            try:
                # Busca en la pagina de contacto
                contact_url = contact_link["href"]
                if not contact_url.startswith('https://') and not contact_url.startswith('http://'): 
                    contact_url = url + contact_url
                contact_response = requests.get(contact_url, timeout=2)
                contact_soup = BeautifulSoup(contact_response.content, "html.parser")
                email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[com|es|net|org|edu|gov|info|biz|io|co|uk|au|ca]{2,}"
                emails = list(set((re.findall(email_pattern, str(contact_soup)))))

                # Filtrado para emails muy largos
                n = 40 
                emails = [x for x in emails if len(x) <= n]
                email_list[-1] = emails
                if email_list[-1] == []: email_list[-1] = ['Posible formulario de contacto']
            except: print('Posible form'); email_list[-1] = ['No se puede acceder a la pagina de contacto']
        print(f'Loading {round(100*(i/(len(web)-1)),2)}%')
        if email_list[-1] == []: email_list[-1] = ['No se ha encontrado el email en la web']
    except Exception as e:  email_list.append(['No se puede acceder a la web']); print(f'Error: {e}')

# Cuenta los mails obtenidos 
count = 0
for i in range(len(email_list)):
    if '@' in email_list[i][0]: count += 1
emails_found = round(100*(1-(count/len(web))),2)

# Display del % de emails y df con la info 
print(f'Emails found: {emails_found}%')
df_emails = pd.DataFrame({'Emails':email_list, 'Webs': web})

Loading 0.0%
Loading 0.78%
Loading 1.56%
Loading 2.34%
Loading 3.12%
Error: HTTPConnectionPool(host='www.tiendaciaobella.es', port=80): Max retries exceeded with url: /%20%20INSTAGRAM-ciao_bella_valencia (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E2243C6FD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Loading 4.69%
Loading 5.47%
Loading 6.25%
Loading 7.03%
Loading 7.81%
Loading 8.59%
Loading 9.38%
Loading 10.16%
Loading 10.94%
Loading 11.72%
Loading 12.5%
Loading 13.28%
Error: HTTPConnectionPool(host='www.marabushop.es', port=80): Read timed out. (read timeout=2)
Loading 14.84%
Error: HTTPSConnectionPool(host='www.koopera.org', port=443): Read timed out. (read timeout=2)
Loading 16.41%
Loading 17.19%
Loading 17.97%
Loading 18.75%
Loading 19.53%
Loading 20.31%
Loading 21.09%
Loading 21.88%
Loading 22.66%
Loading 23.44%
Loading 24.22%
Loading 25.0%
Loading 25.78%
Loading 26.56%
Loading 27.34%
Loading 28.12%
Loading 2